In [1]:
import sqlite3
import json
from typing import Literal, Generator, List, Optional
from sqlite3 import Error, PrepareProtocol, register_adapter

In [ ]:
from pydantic import BaseModel

class Message(BaseModel):
    id: int = 0
    role: Literal["system", "user", "assistant"]
    content: str
    summary_id: int = 0

class Summary(BaseModel):
    id: int
    title: str
    summary: str
    entities: List[str]
    
class SummaryAndMessages(BaseModel):
    summary: Summary
    messages: List[Message]


In [ ]:
conn = sqlite3.connect(r'Z:\electron_groq\database.db')
# conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [ ]:
conn.close()

In [1]:
%load_ext sql
%sql sqlite:///Z:/electron_groq/database.db

Connecting to 'sqlite:///Z:/electron_groq/database.db'

In [3]:
%%sql
SELECT * FROM sqlite_master;

Running query in 'sqlite:///Z:/electron_groq/database.db'

type,name,tbl_name,rootpage,sql
table,summary,summary,3,"CREATE TABLE summary ( id integer PRIMARY KEY, title text NOT NULL, summary text NOT NULL, entities text NOT NULL )"
table,master,master,4,"CREATE TABLE master (id, role, content, summary_id)"
table,temp_chat,temp_chat,2,"CREATE TABLE temp_chat ( id integer PRIMARY KEY, role text NOT NULL, content text NOT NULL )"


In [5]:
%%sql 
SELECT * FROM temp_chat LIMIT 2;

Running query in 'sqlite:///Z:/electron_groq/database.db'

id,role,content
1,system,"You are **SuperModels**, an ingenious human-computer interface that combines all the best AI models out there. You are fast, eloquent, and succinct. And you incorporate various research and analysis tools in your workflow. Furthermore, you have agency."
2,user,what can you ddo?


In [ ]:
cursor.execute("""CREATE TABLE IF NOT EXISTS master_s (id, role, content);""")

cursor.execute("""
    INSERT INTO master_s (id, role, content)
    SELECT id, role, content FROM master
    UNION ALL
    SELECT id, role, content FROM temp_chat;
""")

cursor.execute("DROP TABLE master")

cursor.execute("CREATE TABLE IF NOT EXISTS master (id, role, content);")

cursor.execute("""
    INSERT INTO master (id, role, content)
    SELECT id, role, content FROM master_s
""")

cursor.execute("DROP TABLE master_s")

conn.commit()

In [ ]:
cursor.execute("SELECT * FROM temp_chat")
rows = cursor.fetchall()
messages = [{"role": row[1], "content": row[2]} for row in rows]  # use pydantic's [message.to_dict() for row in rows]
messages

In [ ]:
row = {"role": rows[0][1], "content": rows[0][2]}
Message(**row)

In [ ]:
from groq import Groq
import os, json

conn.row_factory = sqlite3.Row
cursor = conn.cursor()

client = Groq(api_key=os.getenv('GROQ_API_KEY')) 

cursor.execute("SELECT * FROM temp_chat")
rows = cursor.fetchall()
messages = [Message(role=row['role'], content=row['content']) for row in rows] 

messages.append(Message(role="system", content=f"You are a summary database that outputs summaries in JSON.\n\
    The JSON object must use the schema: {json.dumps(Summary.model_json_schema(), indent=2)}"))
    
messages.append(Message(role="user", content="Summarise the above conversation and list up all named entities."))

main_request = client.chat.completions.create(model="mixtral-8x7b-32768",
                                              messages=messages,
                                              response_format={"type": "json_object"})
         
res = main_request.choices[0].message.content

summary_dict = Summary.model_validate_json(res).model_dump()

In [ ]:
summary_dict['summary']

In [ ]:
cursor.execute("""SELECT * FROM summary;""")
res = cursor.fetchall()
res

In [ ]:
cursor.execute("""SELECT MAX(id) FROM summary;""")
res = cursor.fetchone()
res[0]

In [ ]:
cursor.execute("DROP TABLE temp_chat")
cursor.execute("DROP TABLE master")
# cursor.execute("DROP TABLE master_s")
cursor.execute("""CREATE TABLE IF NOT EXISTS master (
    id integer, role text, content text, summary_id integer);""")

cursor.execute("DROP TABLE summary")
cursor.execute("""CREATE TABLE IF NOT EXISTS summary (
                            id integer PRIMARY KEY,
                            title text NOT NULL,
                            summary text NOT NULL, 
                            entities text NOT NULL
                            );""")
conn.commit()

In [ ]:
conn.close()

### using [adaptors](https://docs.python.org/3/library/sqlite3.html#how-to-adapt-custom-python-types-to-sqlite-values) to conform objects.

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y

    def __conform__(self, protocol):
        if protocol is sqlite3.PrepareProtocol:
            return f"{self.x};{self.y}"

con = sqlite3.connect(":memory:")
cur = con.cursor()

cur.execute("SELECT ?", (Point(4.0, -3.2),))
print(cur.fetchone()[0])

In [ ]:
def adapt(summary):
    return ' '.join(summary.entities)
    
register_adapter(Summary, adapt)

In [ ]:
def adapt_list_to_json(lst):
    return json.dumps(lst).encode('utf-8')

def convert_json_to_list(text):
    return json.loads(text.decode('utf-8'))

sqlite3.register_adapter(list, adapt_list_to_json)
sqlite3.register_converter("JSON", convert_json_to_list)

# Connect to your SQLite database, enabling the converter
conn = sqlite3.connect('database.db', detect_types=sqlite3.PARSE_DECLTYPES)
cursor = conn.cursor()

cursor.execute("""
            INSERT INTO summary (summary, entities) 
            VALUES (?, ?);""", (summary.summary, summary.entities))

# Don't forget to commit and close
conn.commit()
conn.close()

In [ ]:
summary.entities

In [ ]:
cursor.execute("DROP TABLE temp_chat")

cursor.execute("""CREATE TABLE IF NOT EXISTS temp_chat (
                                id integer PRIMARY KEY,
                                role text NOT NULL,
                                content text NOT NULL                                
                            );""")
conn.commit()

In [ ]:
# add a column
summary_id = 1
cursor.execute(f"""
               ALTER TABLE temp_chat
               ADD COLUMN summary_id integer DEFAULT {summary_id};""")
conn.commit()
# cursor.fetchall()

In [ ]:
# set a value
summary_id = 1 
cursor.execute("""
UPDATE temp_chat
SET summary_id = ?;
""", (summary_id,))

In [ ]:
cursor.execute("DROP TABLE master")
cursor.execute("""CREATE TABLE IF NOT EXISTS master (
    id integer PRIMARY KEY, 
    role text, 
    content text, 
    summary_id integer);""")

cursor.execute("DROP TABLE summary")
cursor.execute("""CREATE TABLE IF NOT EXISTS summary (
                            id integer PRIMARY KEY,
                            summary text NOT NULL, 
                            entities text NOT NULL
                            );""")
conn.commit()

In [ ]:
cursor.execute("""CREATE TABLE IF NOT EXISTS temp_chat (
                                id integer PRIMARY KEY,
                                role text NOT NULL,
                                content text NOT NULL                                
                            );""")
conn.commit()

In [ ]:
%%sql
SELECT COUNT(*) FROM temp_chat;

In [ ]:
%%sql
SELECT * FROM summary;
-- WHERE id == 1;

In [ ]:
cursor.execute("""SELECT * FROM summary 
               WHERE id == ?;""", (1,))
conn.commit()

In [ ]:
class YesOrNo(BaseModel):
    answer: bool
    reason: str
    userInputRequired: Optional[bool]

In [ ]:
YesOrNo.model_json_schema()